# Explain an Intermediate Layer of VGG16 on ImageNet

Explaining a prediction in terms of the original input image is harder than explaining the predicition in terms of a higher convolutional layer (because the higher convolutional layer is closer to the output). This notebook gives a simple example of how to use GradientExplainer to do explain a model output with respect to the 7th layer of the pretrained VGG16 network.

Note that by default 200 samples are taken to compute the expectation. To run faster you can lower the number of samples per explanation.

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1.keras.backend as K

tf.compat.v1.disable_eager_execution()
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
import shap
# import keras.backend as K
import json


# explain how the input to the 7th layer of the model explains the top two classes
def map2layer(x, layer):
    # model.layers[0].input contains the input shape into the model (224,224,3)
    # preprocess_input(X) contains a image
    feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))  # zip combines the to arrays [a,b,c] , [1,2,3] -> [(a,1),(b,2),(c,3)] - dict creates a dict from the input
    return K.get_session().run(model.layers[layer].input, feed_dict)


# load pre-trained model and choose two images to explain
model = VGG16(weights='imagenet', include_top=True)
print(model.layers)
X, y = shap.datasets.imagenet50()
# we need to load the pictures manual
# y <- apple : 948 , strawberry : 949

file1 = "/home/jan/shap/notebooks/kernel_explainer/data/apple-with-grass.jpg"
img1 = image.load_img(file1, target_size=(224, 224))
img_orig1 = image.img_to_array(img1)

file2 = "/home/jan/shap/notebooks/kernel_explainer/data/strawberry-with-grass.jpg"
img2 = image.load_img(file2, target_size=(224, 224))
img_orig2 = image.img_to_array(img2)

# to_explain = X[[39, 41]] (2, 224, 224, 3)

to_explain = array([img_orig1, img_orig2])

# load the ImageNet class names
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
fname = shap.datasets.cache(url)
with open(fname) as f:
    class_names = json.load(f)

e = shap.GradientExplainer((model.layers[7].input, model.layers[-1].output), map2layer(preprocess_input(X.copy()), 7))
stuff = map2layer(to_explain, 7)
shap_values, indexes = e.shap_values(stuff, ranked_outputs=2)

# get the names for the classes
index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)

# plot the explanations
shap.image_plot(shap_values, to_explain, index_names)

Using TensorFlow backend.


[<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f9b65d0b2b0>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9ba126f898>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9b64939dd8>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f9b64904be0>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9b64904f98>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9b64918cc0>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f9b648a66d8>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9b648a6dd8>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9b648b7ac8>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9b648ce908>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f9b648d44e0>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9b648d49b0>, <tensorflow.python.keras.layers.convo

## Explain with local smoothing

Gradient explainer uses expected gradients, which merges ideas from integrated gradients, SHAP, and SmoothGrad into a single expection equation. To use smoothing like SmoothGrad just set the local_smoothing parameter to something non-zero. This will add normally distributed noise with that standard deviation to the input during the expectation calculation. It can create smoother feature attributions that better capture correlated regions of the image.

In [ ]:
# explain how the input to the 7th layer of the model explains the top two classes
explainer = shap.GradientExplainer(
    (model.layers[7].input, model.layers[-1].output),
    map2layer(preprocess_input(X.copy()), 7),
    local_smoothing=100
)
shap_values,indexes = explainer.shap_values(map2layer(to_explain, 7), ranked_outputs=2)

# get the names for the classes
index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)

# plot the explanations
shap.image_plot(shap_values, to_explain, index_names)